In [ ]:
import pandas   as pd
import polars   as pl
import numpy    as np
import networkx as nx
import time
import sys
sys.path.insert(1, '../rtsvg')
from rtsvg import *
rt = RACETrack()


_figures_out_initial_filter_ = '''
g_nx = rt.createNetworkXGraph(df, [('sbj','obj','vrb')])
print(f'{len(g_nx)=} | {len(g_nx.edges)=}')
to_keep = set(nx.shortest_path(g_nx, _src_, _dst_))
_path_  = nx.shortest_path(g_nx, _src_, _dst_)
for i in range(len(_path_)-1): 
    g_nx.remove_edge(_path_[i],_path_[i+1])
print(f'{len(g_nx)=} | {len(g_nx.edges)=} | {len(to_keep)=}')
for n0 in _path_:
    for n1 in _path_:
        if n0 == n1: continue
        try:
            _p_ = nx.shortest_path(g_nx, n0, n1)
            to_keep = to_keep | set(_p_)
        except:
            pass
print(f'{len(to_keep)=}')
'''

_filter_op_ = '''
_filter_ = {467458, 2983428, 352774, 5190151, 956937, 1873418, 1873420, 610830, 4527122, 2225684, 3892244, 5252128, 3370535, 3588136, 
            6330927, 233519, 1389103, 1461815, 324153, 6438969, 1891900, 2176575, 1937990, 2799176, 120910, 245839, 3163219, 7760468, 
            770644, 116822, 649307, 264295, 335976, 335977, 329840, 7055472, 1860218, 492671, 5746830, 1862294, 499866, 407200, 121508, 
            501419, 6417068, 5052588, 2226863, 4174018, 1621189, 467663, 6429909, 406744, 226522, 490203, 410845, 214749, 6302444, 
            1391343, 1369338, 611067, 3402499, 6714118, 212231, 503566, 215321, 6438681, 6331163, 6809887, 3147039, 603944, 5917483, 
            1583412, 5259071, 249665, 5902660, 6729546, 2452812, 439628, 256336, 4828499, 214869, 2642781, 2390373, 316774, 324089, 
            6729582, 927087, 5343089, 7745394, 1199987, 631157, 497526, 496503, 645505, 241539, 3066244, 514437, 6646662, 2271624, 
            601481, 742283, 2223509, 5339543, 406938, 3559324, 4130717, 500126, 514469, 1939878, 1939879, 1713576, 5889449, 1939880, 
            5339561, 5953968, 2444210, 310198, 6327734, 6305720, 241593, 1371586, 214980, 2223559, 4529097, 333771, 8009169, 767961, 
            332252, 4862943, 121318, 3146727, 1193960, 5346281, 7494636, 6156782, 295408, 982000, 5339637, 146425, 1889786, 7760382}

ofi  = rt.ontologyFrameworkInstance(base_filename='../../data/kaggle_imdb_600k/imdb_600k_international_movies')
print(f'Original size: {len(ofi.df_triples)}')
df   = ofi.df_triples.filter((pl.col('sbj').is_in(_filter_)) | (pl.col('obj').is_in(_filter_)))
print(f'Initial Filtered Dataframe: {len(df)}')
print(f'Original Filter: {len(_filter_)}')
for _expansion_ in range(4):
    _filter_expand_by_one_ = set(df['sbj']) | set(df['obj'])
    print(f'{_expansion_} Expanded Filter: {len(_filter_expand_by_one_)}')
    ofi  = rt.ontologyFrameworkInstance(base_filename='../../data/kaggle_imdb_600k/imdb_600k_international_movies')
    df   = ofi.df_triples.filter((pl.col('sbj').is_in(_filter_expand_by_one_)) | (pl.col('obj').is_in(_filter_expand_by_one_)))
    print(f'{_expansion_} Expanded Dataframe: {len(df)}')
'''
#ofi  = rt.ontologyFrameworkInstance(base_filename='../../data/kaggle_imdb_600k/20240519_ontology')
#df   = ofi.df_triples.unique()
#print(f'{df.shape=}')
#g_nx = rt.createNetworkXGraph(df, [('sbj','obj','vrb')]) # full graph ... this takes 45.2s for ~8m records
#nx.shortest_path(g_nx, 1632498, 3901566)
#_src_, _dst_ = 1632498, 3901566

In [ ]:
df = pl.DataFrame({'sbj':[1, 2, 3, 4, 5, 6,  2,  20, 21, 22, 23, 1,  11, 12, 13, 14, 15, 16, 17, 5,  50, 50],
                   'obj':[2, 3, 4, 5, 6, 7,  20, 21, 22, 23, 4,  11, 12, 13, 14, 15, 16, 17, 5,  50, 4,  4],
                   'vrb':[1, 1, 1, 1, 1, 1,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  5,  5,  6]})
_src_, _dst_ = 1, 7
lns = rt.linkNodeShortest(df, relationships=[('sbj','obj','vrb')], pairs=[(_src_,_dst_), (20, 23), (11, 17, 3, 4)], max_degree_to_show=2,
                          link_color='relationship', link_size='large',
                          node_labels={2:'long string goes here'}, node_color='label', draw_labels=True, max_label_w=64)
lns

In [ ]:
lns.nodes_rendered

In [ ]:
rt.annotateEntities(lns, 1)

In [ ]:
df = pl.DataFrame({'fm':'a  a0 a1 a2 a3 a  b0 b1 b2 b3 b1 c1 c2 c3 c4 c5 c6 c1  a  d0 d1 d2 d3 a  e0 e1 e2 e3 e4 d5 d1 f1 f2 d1 g1 g2'.split(),
                   'to':'a0 a1 a2 a3 b  b0 b1 b2 b3 b  c1 c2 c3 c4 c5 c6 b  a   d0 d1 d2 d3 b  e0 e1 e2 e3 e4 e5 a1 f1 f2 d2 g1 g2 d2'.split()})
lns = rt.linkNodeShortest(df, relationships=[('fm','to')], pairs=('a','b'), max_degree_to_show=2,
                          node_color='label', draw_labels=True, max_label_w=64)
lns

In [ ]:
STOP STOP STOP
ofi  = rt.ontologyFrameworkInstance(base_filename='../../data/kaggle_imdb_600k/20240519_ontology')
df   = ofi.df_triples.unique()
print(f'{df.shape=}')
#g_nx = rt.createNetworkXGraph(df, [('sbj','obj','vrb')]) # full graph ... this takes 45.2s for ~8m records
#nx.shortest_path(g_nx, 1632498, 3901566)
_src_, _dst_ = 1632498, 3901566
lns = rt.linkNodeShortest(df, relationships=[('sbj','obj','vrb')], pairs=(_src_,_dst_), draw_labels=True, node_color='label')
lns

In [ ]:
lns.nodes_rendered

In [ ]:
g = nx.Graph()
g.add_edge('a','b'), g.add_edge('a','c'), g.add_edge('a','d'), g.add_edge('d','e'), g.add_edge('f','d')
g.degree['a'], g.degree['b'], g.degree['c'], g.degree['d']

In [ ]:
lns.time_lu